In [ ]:
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import spatial

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from csv import DictReader
import csv
import os
import re
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn import feature_extraction
from tqdm import tqdm
from collections import Counter
import operator
import pandas as pd
import string
import preprocessor as p
from nltk import bigrams, trigrams
from collections import defaultdict
import multiprocessing as mp,os

In [ ]:
#File_Names_List = ['wtmp.log', 'cups_access.log', 'lightdm_seat0-greeter.log', 'syslog.log', 'filebeat.log', 'fontconfig.log', 'lastlog.log', 'cups_page.log', 'cloud-init-output.log', 'packetbeat.log', 'ufw.log', 'alternatives.log', 'kern.log', 'lightdm_x-0.log', 'auth_temp.log', 'cups_error.log', 'apt_history.log', 'gpu-manager.log', 'dpkg.log', 'fsck_checkroot.log', 'cloud-init.log', 'auth.log', 'btmp.log', 'apport.log', 'Xorg.0.log', 'lightdm_lightdm.log', 'apt_term.log', 'unattended-upgrades-shutdown.log', 'unattended-upgrades.log', 'dhcp_hostname.log', 'fsck_checkfs.log']
File_Names_List = ["auth.log","kern.log","syslog.log"] 
#File_Names_List = ["cups_access.log"]
#File_Names_List = ["kern.log"]
###import glob, os
###File_Names_List = []
###os.chdir("/home/ubuntu/logs/")
###for file in glob.glob("*.log"):
###    File_Names_List.append(file)
###print(File_Names_List)


In [ ]:
#"/home/ubuntu/logs/cups_access.log"
def process(line,temp_words_l1):
    temp_words_l0=[]
    cnt = 0
    for term in re.findall(r"(?:[a-z][a-z'\-_]+[a-z])",line.lower()):
        cnt += 1
        if cnt < 3:
            continue
        else:
            temp_words_l0.append(term)
    
    temp_words_l1.append(temp_words_l0)
    #print(temp_words_l1)
    return temp_words_l1

def process_wrapper(chunkStart, chunkSize,abs_file_path):
    with open(abs_file_path) as f:
        temp_words_l2=[]
        temp_words_l1=[]
        result_l2 = []
        f.seek(chunkStart)
        lines = f.read(chunkSize).splitlines()
        for line in lines:
            result_l2 = process(line,temp_words_l1)
        #print(result_l2)
        temp_words_l2.extend(result_l2)
        return temp_words_l2

def chunkify(fname,size=4096):
    fileEnd = os.path.getsize(fname)
    with open(fname,'rb') as f:
        chunkEnd = f.tell()
        while True:
            chunkStart = chunkEnd
            f.seek(size,1)
            f.readline()
            #print(var_1)
            #print(var_2)
            chunkEnd = f.tell()
            #print("Chunk Start: %s Chunk End: %s Size: %s" %(chunkStart,chunkEnd,chunkEnd-chunkStart))
            yield chunkStart, chunkEnd - chunkStart
            if chunkEnd > fileEnd:
                break

                
cores=90
#bag_of_words = []
file_dir="/home/ubuntu/logs/"

#init objects
pool = mp.Pool(cores)
jobs = []

for file in File_Names_List:
    abs_file_path = os.path.join(file_dir, file)
    #create jobs
    for chunkStart,chunkSize in chunkify(abs_file_path):
        #temp_words = []
        #print("Chunk-Start: %s Chunk-Size: %s" %(chunkStart,chunkSize))
        jobs.append( pool.apply_async(process_wrapper,(chunkStart,chunkSize,abs_file_path)) )
#        bag_of_words.append(temp_words)

# Get process results from the output queue
#results = [job.get() for job in jobs]

print("============== RESULTS =============")
results=[]
#wait for all jobs to finish
for job in jobs:
    job.wait()
    #print(len(job.get()))
    job_result = job.get()
    results.extend(job_result)
    #print(len(results))


#clean up
pool.close()
#pool.join()





In [ ]:
counter=0
for job in jobs:
    counter+=1
print(counter)
print(len(results))       

In [ ]:
import pickle
with open('results_rev1.pkl', 'wb') as f:
    pickle.dump(results, f)

    
#vocabulary_to_load = pickle.load(open(dictionary_filepath, 'r')) 
#loaded_vectorizer = sklearn.feature_extraction.text.countvectorizer(ngram_range=(ngram_size,ngram_size), 
#min_df=1, vocabulary=vocabulary_to_load) 
#loaded_vectorizer._validate_vocabulary() 
#print('loaded_vectorizer.get_feature_names(): {0}'.   
#format(loaded_vectorizer.get_feature_names())) 

In [3]:
file = open("results_rev1.pkl",'rb')
object_file = pickle.load(file)

results = []
for item in object_file:
    results.append(item)

NameError: name 'pickle' is not defined

In [ ]:
import pprint
pp = pprint.PrettyPrinter(depth=1)
[' '.join(result) for result in results[0:10]]

In [ ]:
#stop_words="english", analyzer='word', ngram_range=(1, 1),
vect = CountVectorizer(max_df=1.0, min_df=1, max_features=20)
X = vect.fit_transform([' '.join(result) for result in results[0:10]])




In [ ]:
print(X.toarray())

In [ ]:
vect.get_feature_names()

In [ ]:
print(type(X.toarray()))
print(X.shape)

In [ ]:
import re

regex_str = [
#    r'<[^>]+>', # HTML tags
#    r'(?:@[\w_]+)', # @-mentions
#    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
#    r'http[s]?://(?:[a-z][$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
#    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
#    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
#    r'(?:[\w_]+)', # other words
#    r'(?:\S)' # anything else
    r'\w+'
]
#words = re.findall(r'\w+', open('hamlet.txt').read().lower())
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize_1(s):
    return tokens_re.findall(s)

def tokenize_2(sent):
    return [x.strip() for x in re.split(r"(?:[a-z][a-z_]+[a-z])", sent) if x.strip()]

In [ ]:
count_results = DataFrame(X, columns=vect.get_feature_names())
print (count_results)

In [ ]:
count_ary = count_results.values
print (count_ary)

In [ ]:
correlation= 1-pairwise_distances(count_ary, metric='cosine')
print (correlation)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlation, vmin=0, vmax=1)
fig.colorbar(cax)
plt.show()

In [ ]:
f = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, stop_words="english", 
                    analyzer='word', ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=20)
Xi = f.fit_transform([' '.join(result) for result in results[0:10]])

In [ ]:
tfid_df = DataFrame(Xi.A, columns=f.get_feature_names())
print (tfid_df)

In [ ]:
tfid_ary = tfid_df.values
print (tfid_ary)

In [ ]:
correlation= 1-pairwise_distances(tfid_ary, metric='cosine')
print (correlation)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlation, vmin=0, vmax=1)
fig.colorbar(cax)
plt.show()